In [1]:
from shared_code.utility.schemas.spark_table_schema import image_table_schema
from shared_code.utility.spark.set_environ import *


os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
set_azure_env()

spark = get_session('add-tokens-from-caption')

In [2]:
spark_df = spark.createDataFrame(spark.read.parquet("D:\\data\\processed\\reddit_images_processed_vit_caption.parquet").collect(),
								 schema=image_table_schema)

print(f"Total Number Of Records From Pre-Processing {spark_df.count()}")
print("== Frame For Caption Token Processing ==")
display(spark_df.limit(10).toPandas())

Total Number Of Records From Pre-Processing 7190
== Frame For Caption Token Processing ==


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,10nlv1e,/data/images/greentext/oq8xv2bgcvea1.jpg,Anon at a family reunion,10nlv1e,Kwispy92,https://i.redd.it/oq8xv2bgcvea1.jpg,None,/r/greentext/comments/10nlv1e/anon_at_a_family...,3e3beee2d82def97e9d753f2b3de7398,greentext,None,True,oq8xv2bgcvea1.jpg,A man in a red shirt and blue jeans is sitting...,/data/images/greentext/thumbnail/oq8xv2bgcvea1...,False
1,training,10nm2yp,D:\data\images\SFWRedheads\rhmaiya3evea1.jpg,In my favorite outfit ✨ what’s yours?,10nm2yp,davine__,https://i.redd.it/rhmaiya3evea1.jpg,NaN,/r/SFWRedheads/comments/10nm2yp/in_my_favorite...,85dad0fe57d2965d9c6091d914b47e3f,SFWRedheads,A little girl in a pink dress is sitting on a ...,False,rhmaiya3evea1.jpg,NaN,None,False
2,training,10nm55b,D:\data\images\sfwpetite\o2xorznywtea1.jpg,Another of the girl from yesterday...,10nm55b,insecure_walrus,https://i.redd.it/o2xorznywtea1.jpg,NaN,/r/sfwpetite/comments/10nm55b/another_of_the_g...,4e60a999943248a425b869eeda650581,sfwpetite,A woman in a bikini and a man in a white tank ...,True,o2xorznywtea1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\o2xorznywte...,False
3,training,10nmlp1,D:\data\images\HotGirlNextDoor\ZBeK9Ra.jpg,Bikini Babes (IKTR),10nmlp1,figurephilosopher,https://i.imgur.com/ZBeK9Ra.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/10nmlp1/bikini_bab...,b98400d9569192d3560e7fae92cefba0,HotGirlNextDoor,A woman in a bikini and a man in a white t-shi...,True,ZBeK9Ra.jpg,two women sitting on top of a sandy beach,D:\data\images\HotGirlNextDoor\thumbnail\ZBeK9...,False
4,training,10nms0d,D:\data\images\CityPorn\mykqut7cjvea1.jpg,Central Park with some fog,10nms0d,silent-skreams,https://i.redd.it/mykqut7cjvea1.jpg,NaN,/r/CityPorn/comments/10nms0d/central_park_with...,8b6caf5a265892152a6af7523a85b1dc,CityPorn,A person in a red jacket is skiing down a snow...,True,mykqut7cjvea1.jpg,a city skyline with a plane flying over it,D:\data\images\CityPorn\thumbnail\mykqut7cjvea...,False
5,training,10nmw96,D:\data\images\CityPorn\d4pxa1d8kvea1.jpg,"Parque Piqueri @ São Paulo, Brazil.",10nmw96,Electrical_Eye_7427,https://i.redd.it/d4pxa1d8kvea1.jpg,NaN,/r/CityPorn/comments/10nmw96/parque_piqueri_sã...,922c54cf1bdd32e76a85973975d8b446,CityPorn,A man in a blue shirt and khaki pants is stand...,True,d4pxa1d8kvea1.jpg,a park filled with trees and shrubs in the mid...,D:\data\images\CityPorn\thumbnail\d4pxa1d8kvea...,False
6,training,10nnaq1,D:\data\images\HotGirlNextDoor\gdfpb17o5uea1.jpg,Burnt Cheeks 😝(IKTR),10nnaq1,insecure_walrus,https://i.redd.it/gdfpb17o5uea1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/10nnaq1/burnt_chee...,ce3cb85a2d06d7260c2e310c1f188a42,HotGirlNextDoor,A little girl in a pink bathing suit is standi...,False,gdfpb17o5uea1.jpg,NaN,None,False
7,training,10nng2e,/data/images/greentext/58i3hv7covea1.jpg,Anon’s special order,10nng2e,KayJay65,https://i.redd.it/58i3hv7covea1.jpg,None,/r/greentext/comments/10nng2e/anons_special_or...,46b6c05ef050a277623a73a716aa84ff,greentext,None,True,58i3hv7covea1.jpg,None,/data/images/greentext/thumbnail/58i3hv7covea1...,False
8,training,10nnrvp,D:\data\images\CityPorn\rn8dtsftqvea1.jpg,"Dublin, Ireland",10nnrvp,user89135,https://i.redd.it/rn8dtsftqvea1.jpg,NaN,/r/CityPorn/comments/10nnrvp/dublin_ireland/,08d9d61765fec72212d0128fab8ec6ac,CityPorn,A group of people walk down the street.,True,rn8dtsftqvea1.jpg,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\rn8dtsftqvea...,False
9,training,10no4io,D:\data\images\SFWRedheads\jNYxEhG.jpg,A cute mirror selfie for you &lt;3,10no4io,Yummy_Cummy,https://i.imgur.com/jNYxEhG.jpg,NaN,/r/SFWRedheads/comments/10no4io/a_cute_mirror_...,4982bbf1e49dcac046d7db82913cf6d2,SFWRedheads,A woman in a white dress is sitting on a couch...,True,jNYxEhG.jpg,a woman sitting on a bed looking at her cell p...,D:\data\imag

In [15]:
import nltk
from nltk import TweetTokenizer
from nltk import sent_tokenize
from collections import OrderedDict
from pyspark.sql.functions import udf

def get_nlk_tokens(text: str):
	try:
		tokenizer = TweetTokenizer()

		first_sentence = sent_tokenize(text)[0]

		# remove numbers and tokenize the text
		tokenized = tokenizer.tokenize(first_sentence.translate({ord(ch): None for ch in '0123456789'}))

		tokenized = [i for i in tokenized if len(i) > 1]

		tokenized = list(OrderedDict.fromkeys(tokenized))

		pos_tagged_text = nltk.pos_tag(tokenized)

		# Extract all nouns, verbs and adverbs and append to the existing
		prompt_keywords = [i[0] for i in pos_tagged_text if i[1][:2] in ['NN', 'VB', 'RB']]

		prompt_keywords = list(OrderedDict.fromkeys(prompt_keywords))

		return pos_tagged_text
	except Exception as e:
		# print(e)
		return None


def get_tokens_for_caption(caption: str):
	try:
		if caption == 'NaN':
			return None
		if caption == "":
			return None
		else:
			tokens = get_nlk_tokens(caption)
			print(tokens)
			return tokens
	except:
		return None

In [17]:
from shared_code.utility.schemas.spark_table_schema import tokenize_caption_schema, \
	image_table_schema_with_caption_tokens

get_caption_image_name_udf = udf(get_tokens_for_caption, tokenize_caption_schema)

updated_captions_collection = spark_df\
	.withColumn("caption_tokens", get_caption_image_name_udf("updated_caption"))\
	.withColumn("text_tokens", get_caption_image_name_udf("text"))\
	.collect()

out_frame = spark.createDataFrame(updated_captions_collection, schema=image_table_schema_with_caption_tokens)

display(out_frame.limit(10).toPandas())

print(f"Total Number Of Records From Pre-Processing {out_frame.count()}")

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated,caption_tokens,text_tokens
0,training,10nlv1e,/data/images/greentext/oq8xv2bgcvea1.jpg,Anon at a family reunion,10nlv1e,Kwispy92,https://i.redd.it/oq8xv2bgcvea1.jpg,None,/r/greentext/comments/10nlv1e/anon_at_a_family...,3e3beee2d82def97e9d753f2b3de7398,greentext,None,True,oq8xv2bgcvea1.jpg,A man in a red shirt and blue jeans is sitting...,/data/images/greentext/thumbnail/oq8xv2bgcvea1...,False,"[(man, NN), (in, IN), (red, JJ), (shirt, NN), ...","[(Anon, NNP), (at, IN), (family, NN), (reunion..."
1,training,10nm2yp,D:\data\images\SFWRedheads\rhmaiya3evea1.jpg,In my favorite outfit ✨ what’s yours?,10nm2yp,davine__,https://i.redd.it/rhmaiya3evea1.jpg,NaN,/r/SFWRedheads/comments/10nm2yp/in_my_favorite...,85dad0fe57d2965d9c6091d914b47e3f,SFWRedheads,A little girl in a pink dress is sitting on a ...,False,rhmaiya3evea1.jpg,NaN,None,False,None,"[(In, IN), (my, PRP$), (favorite, JJ), (outfit..."
2,training,10nm55b,D:\data\images\sfwpetite\o2xorznywtea1.jpg,Another of the girl from yesterday...,10nm55b,insecure_walrus,https://i.redd.it/o2xorznywtea1.jpg,NaN,/r/sfwpetite/comments/10nm55b/another_of_the_g...,4e60a999943248a425b869eeda650581,sfwpetite,A woman in a bikini and a man in a white tank ...,True,o2xorznywtea1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\o2xorznywte...,False,"[(beautiful, JJ), (young, JJ), (woman, NN), (i...","[(Another, DT), (of, IN), (the, DT), (girl, NN..."
3,training,10nmlp1,D:\data\images\HotGirlNextDoor\ZBeK9Ra.jpg,Bikini Babes (IKTR),10nmlp1,figurephilosopher,https://i.imgur.com/ZBeK9Ra.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/10nmlp1/bikini_bab...,b98400d9569192d3560e7fae92cefba0,HotGirlNextDoor,A woman in a bikini and a man in a white t-shi...,True,ZBeK9Ra.jpg,two women sitting on top of a sandy beach,D:\data\images\HotGirlNextDoor\thumbnail\ZBeK9...,False,"[(two, CD), (women, NNS), (sitting, VBG), (on,...","[(Bikini, NNP), (Babes, NNP), (IKTR, NNP)]"
4,training,10nms0d,D:\data\images\CityPorn\mykqut7cjvea1.jpg,Central Park with some fog,10nms0d,silent-skreams,https://i.redd.it/mykqut7cjvea1.jpg,NaN,/r/CityPorn/comments/10nms0d/central_park_with...,8b6caf5a265892152a6af7523a85b1dc,CityPorn,A person in a red jacket is skiing down a snow...,True,mykqut7cjvea1.jpg,a city skyline with a plane flying over it,D:\data\images\CityPorn\thumbnail\mykqut7cjvea...,False,"[(city, NN), (skyline, NN), (with, IN), (plane...","[(Central, NNP), (Park, NNP), (with, IN), (som..."
5,training,10nmw96,D:\data\images\CityPorn\d4pxa1d8kvea1.jpg,"Parque Piqueri @ São Paulo, Brazil.",10nmw96,Electrical_Eye_7427,https://i.redd.it/d4pxa1d8kvea1.jpg,NaN,/r/CityPorn/comments/10nmw96/parque_piqueri_sã...,922c54cf1bdd32e76a85973975d8b446,CityPorn,A man in a blue shirt and khaki pants is stand...,True,d4pxa1d8kvea1.jpg,a park filled with trees and shrubs in the mid...,D:\data\images\CityPorn\thumbnail\d4pxa1d8kvea...,False,"[(park, NN), (filled, VBN), (with, IN), (trees...","[(Parque, NNP), (Piqueri, NNP), (São, NNP), (P..."
6,training,10nnaq1,D:\data\images\HotGirlNextDoor\gdfpb17o5uea1.jpg,Burnt Cheeks 😝(IKTR),10nnaq1,insecure_walrus,https://i.redd.it/gdfpb17o5uea1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/10nnaq1/burnt_chee...,ce3cb85a2d06d7260c2e310c1f188a42,HotGirlNextDoor,A little girl in a pink bathing suit is standi...,False,gdfpb17o5uea1.jpg,NaN,None,False,None,"[(Burnt, NNP), (Cheeks, NNP), (IKTR, NNP)]"
7,training,10nng2e,/data/images/greentext/58i3hv7covea1.jpg,Anon’s special order,10nng2e,KayJay65,https://i.redd.it/58i3hv7covea1.jpg,None,/r/greentext/comments/10nng2e/anons_special_or...,46b6c05ef050a277623a73a716aa84ff,greentext,None,True,58i3hv7covea1.jpg,None,/data/images/greentext/thumbnail/58i3hv7covea1...,False,None,"[(Anon, NNP), (special, JJ), (order, NN)]"
8,training,10nnrvp,D:\data\images\CityPorn\rn8dtsftqvea1.jpg,"Dublin,

Total Number Of Records From Pre-Processing 7190


In [18]:
display(out_frame.where('size(caption_tokens) > 0').limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated,caption_tokens,text_tokens
0,training,10nlv1e,/data/images/greentext/oq8xv2bgcvea1.jpg,Anon at a family reunion,10nlv1e,Kwispy92,https://i.redd.it/oq8xv2bgcvea1.jpg,None,/r/greentext/comments/10nlv1e/anon_at_a_family...,3e3beee2d82def97e9d753f2b3de7398,greentext,None,True,oq8xv2bgcvea1.jpg,A man in a red shirt and blue jeans is sitting...,/data/images/greentext/thumbnail/oq8xv2bgcvea1...,False,"[(man, NN), (in, IN), (red, JJ), (shirt, NN), ...","[(Anon, NNP), (at, IN), (family, NN), (reunion..."
1,training,10nm55b,D:\data\images\sfwpetite\o2xorznywtea1.jpg,Another of the girl from yesterday...,10nm55b,insecure_walrus,https://i.redd.it/o2xorznywtea1.jpg,NaN,/r/sfwpetite/comments/10nm55b/another_of_the_g...,4e60a999943248a425b869eeda650581,sfwpetite,A woman in a bikini and a man in a white tank ...,True,o2xorznywtea1.jpg,a beautiful young woman in a bikini posing for...,D:\data\images\sfwpetite\thumbnail\o2xorznywte...,False,"[(beautiful, JJ), (young, JJ), (woman, NN), (i...","[(Another, DT), (of, IN), (the, DT), (girl, NN..."
2,training,10nmlp1,D:\data\images\HotGirlNextDoor\ZBeK9Ra.jpg,Bikini Babes (IKTR),10nmlp1,figurephilosopher,https://i.imgur.com/ZBeK9Ra.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/10nmlp1/bikini_bab...,b98400d9569192d3560e7fae92cefba0,HotGirlNextDoor,A woman in a bikini and a man in a white t-shi...,True,ZBeK9Ra.jpg,two women sitting on top of a sandy beach,D:\data\images\HotGirlNextDoor\thumbnail\ZBeK9...,False,"[(two, CD), (women, NNS), (sitting, VBG), (on,...","[(Bikini, NNP), (Babes, NNP), (IKTR, NNP)]"
3,training,10nms0d,D:\data\images\CityPorn\mykqut7cjvea1.jpg,Central Park with some fog,10nms0d,silent-skreams,https://i.redd.it/mykqut7cjvea1.jpg,NaN,/r/CityPorn/comments/10nms0d/central_park_with...,8b6caf5a265892152a6af7523a85b1dc,CityPorn,A person in a red jacket is skiing down a snow...,True,mykqut7cjvea1.jpg,a city skyline with a plane flying over it,D:\data\images\CityPorn\thumbnail\mykqut7cjvea...,False,"[(city, NN), (skyline, NN), (with, IN), (plane...","[(Central, NNP), (Park, NNP), (with, IN), (som..."
4,training,10nmw96,D:\data\images\CityPorn\d4pxa1d8kvea1.jpg,"Parque Piqueri @ São Paulo, Brazil.",10nmw96,Electrical_Eye_7427,https://i.redd.it/d4pxa1d8kvea1.jpg,NaN,/r/CityPorn/comments/10nmw96/parque_piqueri_sã...,922c54cf1bdd32e76a85973975d8b446,CityPorn,A man in a blue shirt and khaki pants is stand...,True,d4pxa1d8kvea1.jpg,a park filled with trees and shrubs in the mid...,D:\data\images\CityPorn\thumbnail\d4pxa1d8kvea...,False,"[(park, NN), (filled, VBN), (with, IN), (trees...","[(Parque, NNP), (Piqueri, NNP), (São, NNP), (P..."
5,training,10nnrvp,D:\data\images\CityPorn\rn8dtsftqvea1.jpg,"Dublin, Ireland",10nnrvp,user89135,https://i.redd.it/rn8dtsftqvea1.jpg,NaN,/r/CityPorn/comments/10nnrvp/dublin_ireland/,08d9d61765fec72212d0128fab8ec6ac,CityPorn,A group of people walk down the street.,True,rn8dtsftqvea1.jpg,a city street filled with lots of tall buildings,D:\data\images\CityPorn\thumbnail\rn8dtsftqvea...,False,"[(city, NN), (street, NN), (filled, VBD), (wit...","[(Dublin, NNP), (Ireland, NNP)]"
6,training,10no4io,D:\data\images\SFWRedheads\jNYxEhG.jpg,A cute mirror selfie for you &lt;3,10no4io,Yummy_Cummy,https://i.imgur.com/jNYxEhG.jpg,NaN,/r/SFWRedheads/comments/10no4io/a_cute_mirror_...,4982bbf1e49dcac046d7db82913cf6d2,SFWRedheads,A woman in a white dress is sitting on a couch...,True,jNYxEhG.jpg,a woman sitting on a bed looking at her cell p...,D:\data\images\SFWRedheads\thumbnail\jNYxEhG.jpg,False,"[(woman, NN), (sitting, VBG), (on, IN), (bed, ...","[(cute, NN), (mirror, NN), (selfie, NN), (for,..."
7,training,10npwer,D:\data\images\SFWRedheads\j5j72eb17wea1.jpg,New member 🥰🤭,10npwer,AskForAbbyS,https://i.redd.it/j5j72eb17wea1.jpg,NaN,/r/SFWRedheads/comments/10npwer/new_member/,bd752993af7f62b7082e6b04c7f57c84,SFWRedheads,A woman in a pink

In [19]:
out_frame.write.parquet("D:\\data\\processed\\reddit_images_processed_with_caption_tokens.parquet", mode='overwrite')